In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
mk=1

# Initial 

In [25]:
import scipy.io
import numpy as np
file_name = '/content/drive/MyDrive/2022_SUMMER/Research_Burger/CODE/TranslatePythonCode/'
mat = scipy.io.loadmat(file_name + 'canadafastfood_resorted.mat')
data = mat['data']#[np.where(mtype_int==1)]
TimePeriod = len(np.unique(data[:,1]))
SamplePath = int(len(data)/TimePeriod)
print(TimePeriod,SamplePath)
print(data[:,0].reshape([TimePeriod,SamplePath]).T)

#% Labeling
MT = len(data)          #% Size of the dataset (M x T = 400 x 35 = 14000).#clusterid = np.int_(data[:,0])      % Unique market ID.
year = np.int_(data[:,1]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_aw  = np.int_(data[:,2]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_bk  = np.int_(data[:,3]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_hvy = np.int_(data[:,4]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_mcd = np.int_(data[:,5]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_wdy = np.int_(data[:,6]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_aw  = np.int_(data[:,7]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_bk  = np.int_(data[:,8]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_hvy = np.int_(data[:,9]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_mcd = np.int_(data[:,10]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_wdy = np.int_(data[:,11]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_aw  = np.int_(data[:,12]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_bk  = np.int_(data[:,13]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_hvy = np.int_(data[:,14]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_mcd = np.int_(data[:,15]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_wdy = np.int_(data[:,16]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_aw  = np.int_(data[:,17]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_bk  = np.int_(data[:,18]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_hvy = np.int_(data[:,19]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_mcd = np.int_(data[:,20]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_wdy = np.int_(data[:,21]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
pop = data[:,23].reshape([TimePeriod,SamplePath]).T.reshape(-1)
val = data[:,24].reshape([TimePeriod,SamplePath]).T.reshape(-1)
inc = data[:,25].reshape([TimePeriod,SamplePath]).T.reshape(-1)
fpop = data[:,27].reshape([TimePeriod,SamplePath]).T.reshape(-1)
finc = data[:,28].reshape([TimePeriod,SamplePath]).T.reshape(-1)
fval = data[:,29].reshape([TimePeriod,SamplePath]).T.reshape(-1)
mktfe = data[:,30].reshape([TimePeriod,SamplePath]).T.reshape(-1) #% Market fixed effect estimates (from 130913_alaToivanenWaterson3_Fixedeffect3quantile.csv/dta)
tertile = np.int_(data[:,31]).reshape([TimePeriod,SamplePath]).T.reshape(-1)       #% Market type initial guess (from 130913_alaToivanenWaterson3_Fixedeffect3quantile.csv/dta)

35 400
[[  1.   1.   1. ...   1.   1.   1.]
 [  2.   2.   2. ...   2.   2.   2.]
 [  3.   3.   3. ...   3.   3.   3.]
 ...
 [398. 398. 398. ... 398. 398. 398.]
 [399. 399. 399. ... 399. 399. 399.]
 [400. 400. 400. ... 400. 400. 400.]]


In [26]:
def quantile(x,p):
  x = np.sort(x)
  n = len(x)
  x = np.hstack([x[0], x,x[-1]])
  i = p*n+1.5
  iu = int(np.ceil(i))
  il = int(np.floor(i))
  d = i-il
  qq = x[il-1]*(1-d)+x[iu-1]*d
  return qq

#% Number of own shops (in state space; capped at 3), from the perspective of each firm
Ni_aw = (N_aw <= 3)*N_aw + (N_aw > 3)*3
Ni_bk = (N_bk <= 3)*N_bk + (N_bk > 3)*3
Ni_hvy = (N_hvy <= 3)*N_hvy + (N_hvy > 3)*3
Ni_mcd = (N_mcd <= 3)*N_mcd + (N_mcd > 3)*3
Ni_wdy = (N_wdy <= 3)*N_wdy + (N_wdy > 3)*3

#% Number of rival shops (in state space; capped at 3), from the perspective of each firm
Nj_aw = N_bk + N_hvy + N_mcd + N_wdy
Nj_bk = N_aw + N_hvy + N_mcd + N_wdy
Nj_hvy = N_aw + N_bk + N_mcd + N_wdy
Nj_mcd = N_aw + N_bk + N_hvy + N_wdy
Nj_wdy = N_aw + N_bk + N_hvy + N_mcd
Nj_aw = (Nj_aw <= 3)*Nj_aw + (Nj_aw > 3)*3     
Nj_bk = (Nj_bk <= 3)*Nj_bk + (Nj_bk > 3)*3
Nj_hvy = (Nj_hvy <= 3)*Nj_hvy + (Nj_hvy > 3)*3
Nj_mcd = (Nj_mcd <= 3)*Nj_mcd + (Nj_mcd > 3)*3
Nj_wdy = (Nj_wdy <= 3)*Nj_wdy + (Nj_wdy > 3)*3

#% Number of pop,val,inc (in state space; capped at 3)
pop25 = quantile(pop,0.25)
val25 = quantile(val,0.25)   
inc25 = quantile(inc,0.25)  
pop50 = quantile(pop,0.5) 
val50 = quantile(val,0.5)   
inc50 = quantile(inc,0.5)  
pop75 = quantile(pop,0.75) 
val75 = quantile(val,0.75)   
inc75 = quantile(inc,0.75)   
disc_pop = np.int_(pop > pop25) + np.int_(pop > pop50) + np.int_(pop > pop75)  
disc_val = np.int_(val > val25) + np.int_(val > val50) + np.int_(val > val75)  
disc_inc = np.int_(inc > inc25) + np.int_( inc > inc50) + np.int_(inc > inc75)

#% Define state variables for each firm
RHS_aw = np.vstack([Ni_aw, Nj_aw, disc_pop,disc_val, disc_inc]).T#;         % State for A & W.      
RHS_bk = np.vstack([Ni_bk, Nj_bk, disc_pop,disc_val, disc_inc]).T#;         % State for Burger King.      
RHS_hvy = np.vstack([Ni_hvy, Nj_hvy, disc_pop,disc_val, disc_inc]).T#;      % State for Harvey's.      
RHS_mcd = np.vstack([Ni_mcd, Nj_mcd, disc_pop, disc_val,disc_inc]).T#;      % State for McDonald's.      
RHS_wdy = np.vstack( [Ni_wdy, Nj_wdy, disc_pop,disc_val, disc_inc]).T#;

In [27]:
stateID_aw = Ni_aw*4**4+Nj_aw*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_bk = Ni_bk*4**4+Nj_bk*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_hvy = Ni_hvy*4**4+Nj_hvy*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_mcd = Ni_mcd*4**4+Nj_mcd*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_wdy = Ni_wdy*4**4+Nj_wdy*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_other = np.hstack([stateID_aw,stateID_bk,stateID_hvy,stateID_wdy])

ai_aw = -1*np.int_(a_aw<0) + 0*np.int_(a_aw==0) + 1*np.int_(a_aw>0)+1
ai_bk = -1*np.int_(a_bk<0) + 0*np.int_(a_bk==0) + 1*np.int_(a_bk>0) +1
ai_hvy = -1*np.int_(a_hvy<0) + 0*np.int_(a_hvy==0) + 1*np.int_(a_hvy>0)+1
ai_mcd = -1*np.int_(a_mcd<0) + 0*np.int_(a_mcd==0) + 1*np.int_(a_mcd>0)+1
ai_wdy = -1*np.int_(a_wdy<0) + 0*np.int_(a_wdy==0) + 1*np.int_(a_wdy>0)+1
ai_other = np.hstack([ai_aw,ai_bk,ai_hvy,ai_wdy])

In [28]:
numstates = 5 # ni,nj,pop,val,inc
states_full = np.arange(4**numstates)
Ni = states_full//(4**(numstates-1))                       #Number of own outlets: {0,1,2,3+}.
Nj = states_full%(4**(numstates-1))//(4**(numstates-2))    #Number of rival outlets: {0,1,2,3+}.
dz1 = states_full%(4**(numstates-2))//(4**(numstates-3))   #Discretized population state: {0,1,2,3}.
dz2 = states_full%(4**(numstates-3))//(4**(numstates-4))   #Discretized property value state: {0,1,2,3}.
dz3 = states_full%(4**(numstates-4))                       #Discretized income state: {0,1,2,3}.
print(np.sum(np.abs(Ni*4**4+Nj*4**3+dz1*4**2+dz2*4**1+dz3-states_full)))
states = np.stack([Ni,Nj,dz1,dz2,dz3],axis=1)

0


# Function

##1. Global transtion F_Ni, F_dz1,F_dz2,F_dz3

In [29]:
I = 2 #MCD and Other homogenous
J = 3 #Action: 0,1,2
NS = 4**(numstates)

In [30]:
#% F_Ni: Own # of shops changes deterministically, as follows
F_Ni = np.zeros([I,J,NS,NS])#; 
F_Ni[0,0,:,:] = np.array([[1]*4**4+[0]*4**4 + [0] *4**4+[0]*4**4]*4**4 +\
                           [[1]*4**4+[0]*4**4 + [0] *4**4+[0]*4**4]*4**4 + \
                           [[0]*4**4+[1]*4**4 + [0] *4**4+[0]*4**4]*4**4 + \
                           [[0]*4**4+[0]*4**4 + [1] *4**4+[0]*4**4]*4**4 )
F_Ni[0,1,:,:] =np.array([[1]*4**4+[0]*4**4 + [0] *4**4+[0]*4**4]*4**4 +\
                           [[0]*4**4+[1]*4**4 + [0] *4**4+[0]*4**4]*4**4 + \
                           [[0]*4**4+[0]*4**4 + [1] *4**4+[0]*4**4]*4**4 + \
                           [[0]*4**4+[0]*4**4 + [0] *4**4+[1]*4**4]*4**4 )
F_Ni[0,2,:,:] = np.array([[0]*4**4+[1]*4**4 + [0] *4**4+[0]*4**4]*4**4 +\
                           [[0]*4**4+[0]*4**4 + [1] *4**4+[0]*4**4]*4**4 + \
                           [[0]*4**4+[0]*4**4 + [0] *4**4+[1]*4**4]*4**4 + \
                           [[0]*4**4+[0]*4**4 + [0] *4**4+[1]*4**4]*4**4 )
F_Ni[1] = F_Ni[0]

disc_pop_mx = disc_pop[tertile==mk].reshape([-1,TimePeriod])
disc_pop_bk = disc_pop_mx[:,0:TimePeriod-1].reshape(-1)
disc_pop_fw = disc_pop_mx[:,1:TimePeriod].reshape(-1)
disc_val_mx = disc_val[tertile==mk].reshape([-1,TimePeriod])
disc_val_bk = disc_val_mx[:,0:TimePeriod-1].reshape(-1)
disc_val_fw = disc_val_mx[:,1:TimePeriod].reshape(-1)
disc_inc_mx = disc_inc[tertile==mk].reshape([-1,TimePeriod])
disc_inc_bk = disc_inc_mx[:,0:TimePeriod-1].reshape(-1)
disc_inc_fw = disc_inc_mx[:,1:TimePeriod].reshape(-1)

#% F_dz: Make simple & intuitive (4x4) versions for exposition purposes
fz1_4x4 = np.zeros([2,4,4])#;
fz2_4x4 = np.zeros([2,4,4])#;
fz3_4x4 = np.zeros([2,4,4])#;

for x0 in range(4):
    for x1 in range(4):
        numer1 = np.sum((disc_pop_fw == x1) * (disc_pop_bk == x0))#;
        numer2 = np.sum((disc_val_fw == x1) * (disc_val_bk == x0))#;
        numer3 = np.sum((disc_inc_fw == x1) * (disc_inc_bk == x0))#;
        denom1 = np.sum((disc_pop_bk == x0))#;
        denom2 = np.sum((disc_val_bk == x0))#;
        denom3 = np.sum((disc_inc_bk == x0))#;

        fz1_4x4[0,x0,x1] = numer1 / denom1#;
        fz2_4x4[0,x0,x1] = numer2 / denom2#;
        fz3_4x4[0,x0,x1] = numer3 / denom3#;
        
        fz1_4x4[1,x0,x1] = numer1
        fz2_4x4[1,x0,x1] = numer2
        fz3_4x4[1,x0,x1] = numer3

F_dz1 = np.kron((fz1_4x4[0,dz1,:])[:,dz1],np.ones([I,J,1,1]))#; % The transition pattern is common across (i,j,mtype)
F_dz2 = np.kron((fz2_4x4[0,dz2,:])[:,dz2],np.ones([I,J,1,1]))#; % The transition pattern is common across (i,j,mtype)
F_dz3 = np.kron((fz3_4x4[0,dz3,:])[:,dz3],np.ones([I,J,1,1]))#; % The transition pattern is common across (i,j,mtype)
print(fz1_4x4)
print(fz2_4x4)
print(fz3_4x4)

[[[9.57337884e-01 4.26621160e-02 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 9.40488842e-01 5.95111583e-02 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 9.49367089e-01 5.06329114e-02]
  [0.00000000e+00 0.00000000e+00 1.15540150e-03 9.98844598e-01]]

 [[5.61000000e+02 2.50000000e+01 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 8.85000000e+02 5.60000000e+01 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 1.20000000e+03 6.40000000e+01]
  [0.00000000e+00 0.00000000e+00 2.00000000e+00 1.72900000e+03]]]
[[[9.25287356e-01 7.47126437e-02 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 9.15611814e-01 8.43881857e-02 0.00000000e+00]
  [0.00000000e+00 2.50626566e-03 9.02255639e-01 9.52380952e-02]
  [0.00000000e+00 0.00000000e+00 1.73357664e-02 9.82664234e-01]]

 [[9.66000000e+02 7.80000000e+01 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 1.08500000e+03 1.00000000e+02 0.00000000e+00]
  [0.00000000e+00 3.00000000e+00 1.08000000e+03 1.14000000e+02]
  [0.00000000e+00 0.00000000e+00 1

## 2. Generate P(nj|policy)

In [ ]:
# % Calculate F from P (& global transition matrices F_Ni, F_dz1, F_dz2) 
def updateF(P):
  F_Nj = np.ones([I,J,NS,NS])#;
  j = 0#;  % F_Nj does not vary with own choice (j = 1:J), so just fill 'j = 1' case & copy
  #% For a given combination of (Ni, dz1, dz2, mtype), make the state indexes (x, x0, x1, x2, x3)
  #% Beliefs: mapping from today's state (Ni,Nj,dz1,dz2) to tomorrow's Nj = {0,1,2,3}
  fnj_mcd = np.zeros([NS,4])#;      % McDonald's belief
  fnj_other = np.zeros([NS,4])#;    % Other 4 chains beliefs
  # for ni in range(4):
  #     for z1 in range(4):
  #         for z2 in range(4):
  nj = 0#;         % If today's # of rivals = 0
  x =  (4**4)*Ni + (4**3)*nj + (4**2)*dz1 + 4*dz2+dz3#;  % index of today's own state
  x0 = (4**4)*0 + (4**3)*Ni + (4**2)*dz1 + 4*dz2+dz3#;  % index of today's state for rivals with 0 shops

  fnj_mcd[x,0] = P[1,1,x0]**4#;                               % Prob(Nj'= 0) from McD's perspective
  #print(np.sum(P[1,1,x0]!=0))

  fnj_mcd[x,1] = 4 * P[1,2,x0] * (P[1,1,x0]**3)#;       % Prob(Nj'= 1) from McD's perspective
  fnj_mcd[x,2] = 6 * (P[1,2,x0]**2) * (P[1,1,x0]**2)#;   % Prob(Nj'= 2) from McD's perspective
  fnj_mcd[x,3] = 4 * (P[1,2,x0]**3) * P[1,1,x0]#;       % Prob(Nj'= 3) from McD's perspective
  
  fnj_other[x,0] = P[0,1,x0] * (P[1,1,x0]**3)#;         % Prob(Nj'= 0) etc. from Others' perspectives
  fnj_other[x,1] = P[0,2,x0] * (P[1,1,x0]**3) + P[0,1,x0] * 3 * P[1,2,x0] * (P[1,1,x0]**2)#;  
  fnj_other[x,2] = P[0,2,x0] * 3 * P[1,2,x0] * (P[1,1,x0]**2) + P[0,1,x0] * 3 * (P[1,2,x0]**2) * P[1,1,x0]
  fnj_other[x,3] = P[0,2,x0] * 3 * (P[1,2,x0]**2) * P[1,1,x0] + P[0,1,x0] * (P[1,2,x0]**3)
  
  nj = 1#;         % If today's # of rivals = 1                               
  x =  (4**4)*Ni + (4**3)*nj + (4**2)*dz1 +4*dz2+dz3#;  % index of today's own state
  x0 = (4**4)*0 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 0 shops
  x1 =  (4**4)*1 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 1 shops
  
  fnj_mcd[x,0] = P[1,0,x1] * (P[1,1,x0]**3)#;
  fnj_mcd[x,1] = P[1,1,x1] * (P[1,1,x0]**3)#;
  fnj_mcd[x,2] = P[1,2,x1] * (P[1,1,x0]**3) + P[1,1,x1] * 3 * P[1,2,x0] * (P[1,1,x0]**2)#;
  fnj_mcd[x,3] = P[1,2,x1] * 3 * P[1,2,x0] * (P[1,1,x0]**2) + P[1,1,x1] * 3 * (P[1,2,x0]**2) * P[1,1,x0]
  
  fnj_other[x,0] = .5 * (P[0,0,x1] * P[1,1,x0]**3) + .5 * (P[0,1,x0] * P[1,0,x1] * (P[1,1,x0]**2))
  fnj_other[x,1] = .5 * (P[0,1,x1] * P[1,1,x0]**3) + .5 * (P[0,1,x0] * P[1,1,x1] * (P[1,1,x0]**2))
  fnj_other[x,2] = .5 * (P[0,2,x1] * (P[1,1,x0]**3) + P[0,1,x1] * 3 * P[1,2,x0] * (P[1,1,x0]**2))+ .5 * (P[0,2,x0] * P[1,1,x1] * (P[1,1,x0]**2) +\
                        P[0,1,x0] * (P[1,2,x1] * (P[1,1,x0]**2) + P[1,1,x1] * 2 * P[1,2,x0] * P[1,1,x0]))
  fnj_other[x,3] = .5 * (P[0,2,x1] * 3 * P[1,2,x0] * (P[1,1,x0]**2)\
                        + P[0,1,x1] * 3 * (P[1,2,x0]**2) * P[1,1,x0])\
                        + .5 * (P[0,2,x0] * P[1,2,x1] * (P[1,1,x0]**2)\
                      + P[0,2,x0] * P[1,1,x1] * 2 * P[1,2,x0] * P[1,1,x0]\
                      + P[0,1,x0] * P[1,2,x1] * 2 * P[1,2,x0] * P[1,1,x0]\
                      + P[0,1,x0] * P[1,1,x1] * (P[1,2,x0]**2))

  nj = 2#;         % If today's # of rivals = 2
  x = (4**4)*Ni + (4**3)*nj + (4**2)*dz1 +4*dz2+dz3#;  % index of today's own state
  x0 = (4**4)*0 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 0 shops
  x1 = (4**4)*1 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 1 shops
  x2 = (4**4)*2 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 2 shops
  
  fnj_mcd[x,0] = (P[1,0,x1]**2) * (P[1,1,x0]**2)
  fnj_mcd[x,1] = (1/3) * (P[1,0,x2] * (P[1,1,x0]**3)) + (2/3) * (P[1,0,x1] * P[1,1,x1] * (P[1,1,x0]**2))
  fnj_mcd[x,2] = (1/3) * (P[1,1,x2] * (P[1,1,x0]**3)) + (2/3) * ((P[1,1,x1]**2) * (P[1,1,x0]**2))
  fnj_mcd[x,3] = (1/3) * (P[1,2,x2] * (P[1,1,x0]**3)  + (P[1,1,x2] * 3 * P[1,2,x0] * (P[1,1,x0]**2)))
  
  fnj_other[x,0] = .5 * 0 + .5 * (P[0,0,x1] * P[1,0,x1] * (P[1,1,x0]**2))
  fnj_other[x,1] = .5 * (P[0,0,x2] * (P[1,1,x0]**3))\
      + .5 * (P[0,0,x1] * P[1,1,x1] * (P[1,1,x0]**2)\
      + P[0,1,x1] * P[1,0,x1] * (P[1,1,x0]**2))
  fnj_other[x,2] = .5 * (P[0,1,x2] * (P[1,1,x0]**3))\
      + .5 * (P[0,1,x1] * P[1,1,x1] * (P[1,1,x0]**2))
  fnj_other[x,3] = .5 * (P[0,2,x2] * (P[1,1,x0]**3)\
      + P[0,1,x2] * 3 * P[1,2,x0] * P[1,1,x0])\
      + .5 * (P[0,2,x1] * P[1,1,x1] * (P[1,1,x0]**2)\
      + P[0,1,x1] * P[1,2,x1] * (P[1,1,x0]**2)\
      + P[0,1,x1] * P[1,1,x1] * 2 * P[1,2,x0] * P[1,1,x0])
  
  nj = 3#;         % If today's # of rivals = 3
  x = (4**4)*Ni + (4**3)*nj + (4**2)*dz1 +4*dz2+dz3#;  % index of today's own state
  x0 = (4**4)*0 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 0 shops
  x1 = (4**4)*1 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 1 shops
  x2 = (4**4)*2 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 2 shops
  x3 = (4**4)*3 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 3 shops
  
  fnj_mcd[x,0] = (2/3) * (P[1,0,x1]**3) * P[1,1,x0]
  fnj_mcd[x,1] = (2/3) * (P[1,0,x1]**2) * P[1,1,x1] + (1/3) * P[1,0,x2] * P[1,0,x1] * (P[1,1,x0]**2)
  fnj_mcd[x,2] = (2/3) * P[1,0,x1] * (P[1,1,x1]**2) + (1/3) * (P[1,0,x2] * P[1,1,x1] * (P[1,1,x0]**2)\
      + P[1,1,x2] * P[1,0,x1] * (P[1,1,x0]**2))
  fnj_mcd[x,3] = (2/3) * (P[1,1,x1]**3) * P[1,1,x0] + (1/3) * P[1,1,x2] * P[1,1,x1] * (P[1,1,x0]**2)
  
  fnj_other[x,0] = .25 * (P[0,0,x1] * (P[1,0,x1]**2) * P[1,1,x0])
  fnj_other[x,1] = .25 * 0 + .25 * (P[0,0,x2] * P[1,0,x1] * (P[1,1,x0]**2))\
      + .25 * (P[0,0,x1] * 2 * P[1,0,x1] * P[1,1,x1] * P[1,1,x0]\
      + P[0,1,x1] * (P[1,0,x1]**2) * P[1,1,x0])\
      + .25 * (P[0,0,x1] * P[1,0,x2] * (P[1,1,x0]**2))
  fnj_other[x,2] = .25 * (P[0,0,x3] * (P[1,1,x0]**3))\
      + .25 * (P[0,0,x2] * P[1,1,x1] * (P[1,1,x0]**2)\
      + P[0,1,x2] * P[1,0,x1] * (P[1,1,x0]**2))\
      + .25 * (P[0,0,x1] * (P[1,1,x1]**2) * P[1,1,x0]\
      + P[0,1,x1] * 2 * P[1,0,x1] * P[1,1,x1] * P[1,1,x0])\
      + .25 * (P[0,0,x1] * P[1,1,x2] * (P[1,1,x0]**2)\
      + P[0,1,x1] * P[1,0,x2] * (P[1,1,x0]**2))
  fnj_other[x,3] = .25 * (P[0,1,x3] * (P[1,1,x0]**3))\
      + .25 * (P[0,1,x2] * P[1,1,x1] * (P[1,1,x0]**2))\
      + .25 * (P[0,1,x1] * (P[1,1,x1]**2) * P[1,1,x0])\
      + .25 * (P[0,1,x1] * P[1,1,x2] * (P[1,1,x0]**2))
              
  Denom_mcd = np.reshape(np.sum(fnj_mcd,axis=1),[-1,1]).dot(np.ones([1,4]))#;         % Sum of each row
  fnj_mcd = fnj_mcd / Denom_mcd#;                 % Make sure each row sums up to 1
  fnj_mcd[np.isnan(fnj_mcd)] = 0#;                    % Replace NaN with 0
  F_Nj[0,j,:,:] = fnj_mcd[:,Nj]

  Denom_other = np.reshape(np.sum(fnj_other,axis=1),[-1,1]).dot(np.ones([1,4]))#;             % Sum of each row
  fnj_other = fnj_other / Denom_other#;                   % Make sure each row sums up to 1
  fnj_other[np.isnan(fnj_other)] = 0#;                        % Replace NaN with 0
  F_Nj[1,j,:,:] = fnj_other[:,Nj]

  #F_Nj[2:5,j,:,:] = np.stack([F_Nj[1,j,:,:] for copy in range(3)],axis=0)#;    % Other 4 chains are symmetric

  for j in range(1,J):    
      F_Nj[:,j,:,:] = F_Nj[:,0,:,:]    
  F = F_Ni * F_Nj * F_dz1 * F_dz2 *F_dz3
  return F
from scipy.special import softmax
P_new = 1/J*np.ones([I,J,NS])#softmax(np.arange(I*J*NS).reshape([I,J,NS]),axis=1)
F_new = updateF(P_new.copy())
#np.sum(F_new!=0)

##3.Value Iteration

In [ ]:
from scipy.special import logsumexp
from scipy.special import softmax

def updatePi(Q):
  P = np.zeros([I,J,NS])#; player, action, states, Market Type
  P[0,:,:] = softmax(Q[0],axis=0)
  P[1,:,:] = softmax(Q[1],axis=0)
  P[np.isnan(P)] = 0#;
  return P
def updateQ(thetaS,Q,F,beta = 0.9):
  Q_new = np.zeros([I,J,NS])
  gamma = 0.5772
  
  thetaS_mcd = np.reshape(thetaS[0:int(len(thetaS)/2)],-1)
  thetaS_other = np.reshape(thetaS[int(len(thetaS)/2):len(thetaS)],-1)
  Q_new[0,2] =  np.stack([Ni,Ni *Ni,Ni*Nj,Ni*dz1,Ni*dz2,Ni*dz3, -1* np.ones(NS)],axis=1).dot(thetaS_mcd)  
  Q_new[0,1] =np.stack([Ni,Ni *Ni,Ni*Nj,Ni*dz1,Ni*dz2,Ni*dz3, 0* np.ones(NS)],axis=1).dot(thetaS_mcd)
  Q_new[0,0] = Q_new[0,1].copy()

  Q_new[1,2] =  np.stack([Ni,Ni *Ni,Ni*Nj,Ni*dz1,Ni*dz2,Ni*dz3, -1* np.ones(NS)],axis=1).dot(thetaS_other)  
  Q_new[1,1] =np.stack([Ni,Ni *Ni,Ni*Nj,Ni*dz1,Ni*dz2,Ni*dz3, 0* np.ones(NS)],axis=1).dot(thetaS_other)
  Q_new[1,0] = Q_new[1,1].copy()
  
  v_mcd = gamma + logsumexp(Q[0],axis=0) #NS_new
  v_other = gamma + logsumexp(Q[1],axis=0) #NS_new
  #v_mcd = v_mcd.reshape([-1,1])
  #v_other =v_other.reshape([-1,1])
  for j in range(J):
    #print(np.shape(F[0,j].dot(v_mcd)),Q_new[0,j].shape)
    Q_new[0,j] = Q_new[0,j] + beta*(F[0,j].dot(v_mcd))
    Q_new[1,j] = Q_new[1,j] + beta*(F[1,j].dot(v_other))
  return Q_new


def ValueIteration(thetaS,error = 5e-2,beta = 0.6):
  Pi0 = 1/3*np.ones([I,J,NS])
  F0= updateF(Pi0)
  Q0 = np.zeros([I,J,NS])
  Pi_old = Pi0
  F_old = F0
  Q_old = Q0
  for iter in range(1000):
    Q_new = updateQ(thetaS,Q_old,F_old,beta=beta) #I, J, NS
    Pi_new = updatePi(Q_new)#I,J,NS
    F_new = updateF(Pi_new) # I, J, NS,NS

    error1 = np.sum(np.abs(Q_new-Q_old))/np.max(np.abs(Q_old))
    error2 = np.sum(np.abs(Pi_new-Pi_old))/np.max(np.abs(Pi_old))
    error3 = np.sum(np.abs(F_new-F_old))/np.max(np.abs(F_old))
    #print('iteration {} --> e1:{},e2:{},e3:{}'.format(iter,error1,error2,error3))
    if error1<error and error2<error and error3<error:
      #print('iteration {} --> e1:{},e2:{},e3:{}'.format(iter,error1,error2,error3))
      break
    Q_old = Q_new.copy()
    Pi_old = Pi_new.copy()
    F_old = F_new.copy()
  return Q_new, Pi_new,F_new
# thetaS = np.array([2.09932422e+01,  9.39319015e-02, -2.55199026e-01,  5.22040762e-02, 3.74165364e-01,  1.85959336e+02,
# 2.07684086, -1.34568641, -0.06689713, -0.09297975, -0.07754167,9.49715754])
# thetaS = np.zeros(7*2)
# Q_new, Pi_new,F_new = ValueIteration(thetaS)

##4. log-likelihood

In [ ]:
tertile_other = np.hstack([tertile,tertile,tertile,tertile])
def func(thetaS):
  _, Pi_new,_ = ValueIteration(thetaS)
  #print(np.unique(a_mcd))
  res = -np.sum(np.log(Pi_new[0,ai_mcd[tertile==mk],stateID_mcd[tertile==mk]]))
  res = res -np.sum(np.log(Pi_new[1,ai_other[tertile_other==mk],stateID_other[tertile_other==mk]]))
  print(thetaS,res)
  return res/5/MT
from scipy.optimize import minimize
thetaS = np.zeros(7*2)
print('start from:',thetaS )
res22 = minimize(func,thetaS)#,tol=1e-3)
print(res22,res22.fun*5*MT) # 3h

start from: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in double_scalars


Streaming output truncated to the last 5000 lines.
  0.06732373  7.45082988] 16105.574270073113
[-0.46523138 -1.43634033 -1.20519505  0.88839681  0.74974659  0.50437639
  6.3018292   2.03284649 -0.50741859 -1.2633664   0.24602223  0.21495285
  0.06732372  7.45082989] 16105.574270620102
[-0.30830797 -1.36690684 -1.23435135  0.92382205  0.70950168  0.47182758
  6.56964115  2.40000709 -0.74528146 -1.12546162  0.17161375  0.26714594
  0.05969805  7.51215099] 16098.66489814974
[-0.30830797 -1.36690684 -1.23435135  0.92382205  0.70950168  0.47182758
  6.56964115  2.40000709 -0.74528146 -1.12546162  0.17161375  0.26714594
  0.05969805  7.51215099] 16098.66489814974
[-0.30830796 -1.36690684 -1.23435135  0.92382205  0.70950168  0.47182758
  6.56964115  2.40000709 -0.74528146 -1.12546162  0.17161375  0.26714594
  0.05969805  7.51215099] 16098.664898245317
[-0.30830797 -1.36690683 -1.23435135  0.92382205  0.70950168  0.47182758
  6.56964115  2.40000709 -0.74528146 -1.12546162  0.17161375  0.26714

In [ ]:
import numpy as np
res1 = res22.x
res2 = res22.fun*5*MT
print(['{:2.6f}'.format(res1[i]) for i in range(len(res1))])
print('{:6.6f}'.format(-res2))

['9.793806', '-6.489731', '-1.617876', '0.412883', '0.585137', '0.301227', '11.067892', '3.005595', '-1.090149', '-0.560673', '0.213491', '0.130171', '0.028432', '7.656504']
-16060.780721
